# Imports

In [1]:
import numpy as np
import pandas as pd
import mdptoolbox
import time as time
import itertools
import csv
from decimal import * #for handling very small numbers
import pdb  #To use pdb, you can simply import it and call pdb.set_trace()
            #at the point in your code where you want the debugger to pause execution. 

In [2]:
# Import this file from (python file)
pre_codes=open(r"/Users/mohammadmaydanchi/Library/CloudStorage/OneDrive-AuburnUniversity/Auburn University/RA/GMS/9-Second Contribution/2-GMS/2-Working/4-Back up codes/When I am in SA/Main Codes/without decimal/Import this file.py",encoding="utf8")
read_pre_codes=pre_codes.read()
exec(read_pre_codes)


# P

## Indicate Values to Alphas and Betas

In [3]:
def get_alpha_beta(key):
    """
    ##
    this function works inside the calculate_formula(key,string) function.
    this function gets a key which iincludes (from_state, action, to_state).
    It first find the inspections based on the from_states and actions.
    Then it find the alphas and betas based on the inspections which are used oon the lines.
    then output is the alphas and betas of each inspection : Inspection A-1, A-2, B-1, B-2, M(this one is alway obvious
    because it is alway sequencing and we have no other options)
    
    """
    #change the name of variable just for making everything simpler
    single_trans=key
    
    #initial values
    inspections_of_transition = {}
    insp_A1='0'
    insp_A2='0'
    insp_B1='0'
    insp_B2='0'
    insp_M='0'
    on_line_A=True
    on_line_B=True
    on_line_M=True

    
    #default values
    from_state, action, to_state = single_trans
    
    line_A_inspection_1 = from_state[0]
    line_A_inspection_2 = from_state[1]
    line_B_inspection_1 = from_state[2]
    line_B_inspection_2 = from_state[3]

    
    line_M_inspection=insp_M
    
    
    
    #main function
    
    #finding the type of inspections for the states
    #before merge line
    if action!='a-M' and action!='a-F' :
        
        #line A
        while on_line_A==True:
            if line_A_inspection_1 == '0':
                 # If no inspection indicated for line A
                insp_A1 = action.split('-')[1][0]
                insp_A2='0'
                on_line_A=False

            elif line_A_inspection_1 != '0' and line_A_inspection_2 =='0' : #and action!='a-M':

                insp_A1=line_A_inspection_1
                insp_A2=action.split('-')[1][0]
                #print(insp_A2)
                on_line_A=False
                
            elif line_A_inspection_1 != '0' and line_A_inspection_2 !='0' :
                insp_A1=line_A_inspection_1
                insp_A2=line_A_inspection_2
                on_line_A=False
        #line B
        while on_line_B==True:
            if line_B_inspection_1 == '0':
                 # If no inspection indicated for line B
                insp_B1 = action.split('-')[1][1]
                insp_B2='0'
                on_line_B=False

            elif line_B_inspection_1 != '0' and line_B_inspection_2 =='0' : #and action!='a-M':
                insp_B1=line_B_inspection_1
                insp_B2=action.split('-')[1][1]
            
                on_line_B=False
                
            elif line_B_inspection_1 != '0' and line_B_inspection_2 !='0' :
                
                insp_B1=line_B_inspection_1
                insp_B2=line_B_inspection_2
                
                on_line_B=False
                
                
                
    #on merge line before finish step            
    elif action =='a-M' and action!='a-F':
        insp_A1=line_A_inspection_1
        insp_A2=line_A_inspection_2
        insp_B1=line_B_inspection_1
        insp_B2=line_B_inspection_2
        insp_M='S'

    #finish step    
    elif action=='a-F':
        insp_A1='unknown'
        insp_A2='unknown'
        insp_B1='unknown'
        insp_B2='unknown'
        insp_M='unknown'
        
        
    inspections_of_transition = {
                'line_A_inspection_1': insp_A1,
                'line_A_inspection_2': insp_A2,

                'line_B_inspection_1': insp_B1,
                'line_B_inspection_2': insp_B2,

                'line_M_inspection':   insp_M
        }
    
    
    #finiding alphas and betas based on inspections
    dict_of_alphas_betas={}
    
    #mapping
    alpha_beta_mapping = {'N': (alpha_N, beta_N), 'G': (alpha_G, beta_G), 'Y': (alpha_Y, beta_Y), 'S': (alpha_S, beta_S),'0':(0,0),'unknown':('unknown','unknown')}
    
    
    for key, value in inspections_of_transition.items():
        alpha_value, beta_value = alpha_beta_mapping.get(value, (None, None))
        dict_of_alphas_betas[key] = (alpha_value, beta_value)

    return dict_of_alphas_betas  #alphas and betas of all the inspection for aparticular transition.
    

## Calculate the probabilities

In [4]:
def calculate_formula(key,string):
    
    """
    It is used in create_P() function
    it gets key and value from transitions , which include all transitions.
    Thsi function uses dict_of_alphas_betas inside itself.
    It gives us the value (float number) for the probability of a transition.
    """
    
    
    #initial values
    value=0
    
    #get all alphas and betas
    dict_of_alphas_betas=get_alpha_beta(key)
    
    alpha_A1=dict_of_alphas_betas['line_A_inspection_1'][0]
    beta_A1=dict_of_alphas_betas['line_A_inspection_1'][1]


    alpha_A2=dict_of_alphas_betas['line_A_inspection_2'][0]
    beta_A2=dict_of_alphas_betas['line_A_inspection_2'][1]


    alpha_B1=dict_of_alphas_betas['line_B_inspection_1'][0]
    beta_B1=dict_of_alphas_betas['line_B_inspection_1'][1]


    alpha_B2=dict_of_alphas_betas['line_B_inspection_2'][0]
    beta_B2=dict_of_alphas_betas['line_B_inspection_2'][1]


    alpha_M=dict_of_alphas_betas['line_M_inspection'][0]
    beta_M=dict_of_alphas_betas['line_M_inspection'][1]
    
    
    #the transition which has happend on all states which are before the final state.
    if beta_M!='unknown':
    
    
        
        
        #the below proability are the main formula for the probabilities.
        #P(IA1p)
        P_IA1p=(1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        #---------------------------------------------------
        #P_IA1f
        P_IA1f=1-P_IA1p
        #---------------------------------------------------
        #P_IB1p
        P_IB1p= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
        #---------------------------------------------------
        #P_IB1f
        P_IB1f=1-P_IB1p
        #---------------------------------------------------
        ##P_IA2p_given_IA1p
        numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
        denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        # Compute the fraction
        fraction = numerator / denominator
        # Add the last term
        P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        #---------------------------------------------------
        ## P_IA2f_given_IA1p
        P_IA2f_given_IA1p=1-P_IA2p_given_IA1p
        #---------------------------------------------------
        ##P_IB2p_given_IB1p
        numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
        denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
        # Compute the fraction
        fraction2 = numerator2 / denominator2
        # Add the last term
        P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        #---------------------------------------------------
        #P_IB2f_given_IB1p
        P_IB2f_given_IB1p=1-P_IB2p_given_IB1p
        #---------------------------------------------------
        #P_IA2p
        P_IA2p=P_IA1p*P_IA2p_given_IA1p
        #---------------------------------------------------
        #P_IB2p
        P_IB2p=P_IB1p*P_IB2p_given_IB1p

        #conditions, the proablity type of a transition., we use this
        #to indicate that for the current transition which above formula we need to use.
        if string[:3] == "P1-":
            #P1
            P1=round(P_IA1p*P_IB1p,11)
            value = P1

        elif string[:3] == "P2-":
            #P2
            P2=round(P_IA1f*P_IB1p,11)
            value = P2

        elif string[:3] == "P3-":   
            #P3
            P3=round(P_IA1p*P_IB1f,11)
            value = P3


        elif string[:3] == "P4-":
            #P4
            #P4=P_IA1f*P_IB1f
            P1=round(P_IA1p*P_IB1p,11)
            P2=round(P_IA1f*P_IB1p,11)
            P3=round(P_IA1p*P_IB1f,11)
            P4=1-(P1+P2+P3)
            value = P4
            
       #--------#----------#----------#-----------#-----------#     

        elif string[:3] == "P5-":
            #P5
            P5=round(P_IA1p*P_IB2p_given_IB1p,11)
            value = P5



        elif string[:3] == "P6-":
            #P6
            P6=round(P_IA1f*P_IB2p_given_IB1p,11)
            value = P6



        elif string[:3] == "P7-":
            #P7
            P7=round(P_IA1p*P_IB2f_given_IB1p,11)
            value = P7    


        elif string[:3] == "P8-":
            #P8
            #P8=P_IA1f*P_IB2f_given_IB1p
            P5=round(P_IA1p*P_IB2p_given_IB1p,11)
            P6=round(P_IA1f*P_IB2p_given_IB1p,11)
            P7=round(P_IA1p*P_IB2f_given_IB1p,11)
            P8=1-(P5+P6+P7)
            value = P8

        #--------#----------#----------#-----------#-----------#  

        elif string[:3] == "P9-":  
            #P9
            P9=round(P_IA2p_given_IA1p*P_IB2p_given_IB1p,11)
            value=P9



        elif string[:3] == "P10":
            #P10
            P10=round(P_IA2f_given_IA1p*P_IB2p_given_IB1p,11)
            value=P10


        elif string[:3] == "P11":
            #P11
            P11=round(P_IA2p_given_IA1p*P_IB2f_given_IB1p,11)
            value=P11    


        elif string[:3] == "P12":
            #P12
            #P12=P_IA2f_given_IA1p*P_IB2f_given_IB1p
            P9=round(P_IA2p_given_IA1p*P_IB2p_given_IB1p,11)
            P10=round(P_IA2f_given_IA1p*P_IB2p_given_IB1p,11)
            P11=round(P_IA2p_given_IA1p*P_IB2f_given_IB1p,11)
            P12=1-(P9+P10+P11)
            value=P12    

        #--------#----------#----------#-----------#-----------#  

        elif string[:3] == "P13":
            #P13
            P13=round(P_IA2p_given_IA1p*P_IB1p,11)
            value=P13   


        elif string[:3] == "P14":
            #P14
            P14=round(P_IA2p_given_IA1p*P_IB1f,11)
            value=P14



        elif string[:3] == "P15":
            #P15
            P15=round(P_IA2f_given_IA1p*P_IB1p,11)
            value=P15   


        elif string[:3] == "P16":
            #P16
            #P16=P_IA2f_given_IA1p*P_IB1f
            P13=round(P_IA2p_given_IA1p*P_IB1p,11)
            P14=round(P_IA2p_given_IA1p*P_IB1f,11)
            P15=round(P_IA2f_given_IA1p*P_IB1p,11)
            P16=1-(P13+P14+P15)
            value=P16

        #--------#----------#----------#-----------#-----------#  
        
        elif string[:3] == "P17":
            #P17
            P17=round(P_IA1p,11)
            value=P17    



        elif string[:3] == "P18":
            #P18
            #P18=P_IA1f
            P17=round(P_IA1p,11)
            P18=1-P17
            value=P18
         #--------#----------#----------#-----------#-----------#  
        elif string[:3] == "P19":
            #P19
            P19=round(P_IB1p,11)
            value=P19


        elif string[:3] == "P20":
            #P20
            #P20=P_IB1f
            P19=round(P_IB1p,11)
            P20=1-P19
            value=P20
        #--------#----------#----------#-----------#-----------#  
        elif string[:3] == "P21":
            #P21
            P21=round(P_IA2p_given_IA1p,11)
            value=P21    


        elif string[:3] == "P22":
            #P22
            #P22=P_IA2f_given_IA1p
            P21=round(P_IA2p_given_IA1p,11)
            P22=1-P21
            value=P22
        #--------#----------#----------#-----------#-----------#  
        elif string[:3] == "P23":
            #P23
            P23=round(P_IB2p_given_IB1p,11)
            value=P23

        elif string[:3] == "P24":
            #P24
            #P24=P_IB2f_given_IB1p
            P23=round(P_IB2p_given_IB1p,11)
            P24=1-P23
            value=P24    
        #--------#----------#----------#-----------#-----------#  
        #P(I3p|IAp, IBp)
        elif string[:3] == "P25":
            #P25
            #6
            term1_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 *z_A1* z_B1 * z_B2
            term1_denominator = P_IA2p * P_IB2p
            term1 = term1_numerator / term1_denominator


            #3+4
            term2_numerator = beta_M * beta_A1 * beta_A2  * beta_B2 * z_A1 * ( beta_B1*z_B1 * (1 - z_B2) + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term2_denominator = P_IA2p * P_IB2p
            term2 = term2_numerator / term2_denominator

            #5+8
            term3_numerator = beta_M * (1 - alpha_A1)*  beta_A2 * beta_B1 * beta_B2 * (1 - z_A1) * z_A2 * (beta_B1*z_B1 + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term3_denominator = P_IA2p * P_IB2p
            term3 = term3_numerator / term3_denominator

            #1+9
            term4_numerator = beta_M *  beta_A2 * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (beta_A1 *z_A1 + (1 - alpha_A1)*(1 - z_A1) * z_A2)
            term4_denominator = P_IA2p * P_IB2p
            term4 = term4_numerator / term4_denominator

            #2+7
            term5_numerator = beta_M *  beta_B2 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) *(beta_B1 *z_B1 +  (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term5_denominator = P_IA2p * P_IB2p
            term5 = term5_numerator / term5_denominator

            #11
            term6_numerator = beta_M * z3 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2)
            term6_denominator = P_IA2p * P_IB2p
            term6 = term6_numerator / term6_denominator


            #10
            term7_numerator =  (1 - alpha_M) * (1 - z3)*(1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) 
            term7_denominator = P_IA2p * P_IB2p
            term7 = term7_numerator / term7_denominator

            # Calculate the total expression
            P25 = round(term1 + term2 + term3 + term4 + term5 + term6 + term7,11)
            value = P25

        elif string[:3] == "P26":
            #P26
            #6
            term1_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 *z_A1* z_B1 * z_B2
            term1_denominator = P_IA2p * P_IB2p
            term1 = term1_numerator / term1_denominator


            #3+4
            term2_numerator = beta_M * beta_A1 * beta_A2  * beta_B2 * z_A1 * ( beta_B1*z_B1 * (1 - z_B2) + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term2_denominator = P_IA2p * P_IB2p
            term2 = term2_numerator / term2_denominator

            #5+8
            term3_numerator = beta_M * (1 - alpha_A1)*  beta_A2 * beta_B1 * beta_B2 * (1 - z_A1) * z_A2 * (beta_B1*z_B1 + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term3_denominator = P_IA2p * P_IB2p
            term3 = term3_numerator / term3_denominator

            #1+9
            term4_numerator = beta_M *  beta_A2 * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (beta_A1 *z_A1 + (1 - alpha_A1)*(1 - z_A1) * z_A2)
            term4_denominator = P_IA2p * P_IB2p
            term4 = term4_numerator / term4_denominator

            #2+7
            term5_numerator = beta_M *  beta_B2 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) *(beta_B1 *z_B1 +  (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term5_denominator = P_IA2p * P_IB2p
            term5 = term5_numerator / term5_denominator

            #11
            term6_numerator = beta_M * z3 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2)
            term6_denominator = P_IA2p * P_IB2p
            term6 = term6_numerator / term6_denominator


            #10
            term7_numerator =  (1 - alpha_M) * (1 - z3)*(1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) 
            term7_denominator = P_IA2p * P_IB2p
            term7 = term7_numerator / term7_denominator

            # Calculate the total expression
            P25 = round(term1 + term2 + term3 + term4 + term5 + term6 + term7,11)

            P26=1-P25
            value = P26
    #--------#----------#----------#-----------#-----------#          
    #the transition which happens on the final state  
    elif beta_M!='unknown':
    
        #initial values
        value=0
        #string[:3] == "P27":
        #P27
        P27=1
        value=P27
        
    return value

## Values of Variables

In [108]:
# Test #####################################
#values of variables
#########################
#non-conforming rates

z_A1 =0.9
z_A2 =0.1
z_B1 =0.9
z_B2 =0.1
z3=0.9


#########################
# alphas and betas
alpha_N =0
beta_N = 1

alpha_G = 0.57 #0.1
beta_G = 0.05 #0.1

alpha_Y = 0.05
beta_Y = 0.05

alpha_S = 0.0001
beta_S = 0.0001

alpha_M = alpha_S
beta_M = beta_S

## Function for making transition probability matrix

In [109]:
############################# Create the Transition Probability Matrix #############################

def create_P():
    """
    In this function we use calculate_formula(key,string) function.
    We want to create transition probability matrix based on the characteristicss of the system for all actions,
    in this function.
    For the out put we first create a trans prob matrix for each action,
    then we concat all the matrix, then we convert them to numpy
    then reshape the trans prob matrix in the form of (A,S,S).
    Then we return this numpy 3 dimenwsions matrix.
    """
    
    #create a dictionary with the key includes (from_state, action, to_state) and the value of the 
    #real number for the probability of this transition.
    final_dict={}
    for key, value in transitions.items():
        string=value['probability']
        final_dict[key]=calculate_formula(key,string)
    ###########################################################
    # last step
    # Concatenate states to get rows and columns
    rows_columns = all_states     

    # Initialize an empty list to store matrices
    matrix_list = []

    #create a transition probability matrix for each action
    for action in action_list:
        matrix = pd.DataFrame(0, index=rows_columns, columns=rows_columns)  # Initialize matrix for each action
        # Fill the matrix with transition probabilities for the given action
        for (from_state,action_key, to_state), value in final_dict.items():
            if action_key == action:
                prob_of_cell = value
                matrix.loc[from_state, to_state]=prob_of_cell

    

        # Check if a row's sum is 0 and assign 1 to the corresponding cell
        for state in rows_columns:
            #print(matrix.loc[state].sum())
            if matrix.loc[state].sum() == 0:
                matrix.loc[state, state] = 1

        # Append the matrix to the list
        matrix_list.append(matrix)

    # Concatenate all matrices in the list
    trans_prob_matrix2 = pd.concat(matrix_list, axis=0) #concat under each other

    # convert pandas to numpy
    array_transProbMatrix2=trans_prob_matrix2.to_numpy()
    P=array_transProbMatrix2.reshape(26,442,442)
    
    return P #trans_prob_matrix2 # P  #

## Creatre Transition Probability Matrix

In [110]:
P=create_P()

# R

In [111]:
#rewards
array_rewards=df_rewards.to_numpy() #convert to numpy
resh_array_rewards=array_rewards.reshape(442,26) #reshape
R=resh_array_rewards
#R

# PyMDP

In [112]:
# Total cost use policy iteration method
pi=mdptoolbox.mdp.PolicyIteration(P, R, discount=0.9999, policy0=None, max_iter=10000,eval_type=0)
#pi.setVerbose()
#pi=mdptoolbox.mdp.ValueIteration(transitions, reward, discount=0.9999, max_iter=100000)
#run the  Policy iteration
#discount=0.9999
pi.run() 

total_cost=pi.V
rounded_total_cost=round(total_cost[0],2)
rounded_total_cost

-37876.44

## Policy

In [113]:
policy=pi.policy #get policies of each for eachs tate
policy_list=list(policy) #make the list of the policies

action_dict = {i: action for i, action in enumerate(action_list)} #make the dictionaries of all actions
policy_actualName= [action_dict[key] for key in policy_list] #actual name(acton) for the policy of eachs tate

# Create a dictionary from the two lists using zip function
dict_policyState = dict(zip(all_states, policy_actualName))
#dict_policyState

In [114]:
print(dict_policyState)

{'0000': 'a-GG', 'N000': 'a-YG', 'G000': 'a-YG', 'Y000': 'a-YG', 'S000': 'a-YG', '00N0': 'a-GY', '00G0': 'a-GY', '00Y0': 'a-GY', '00S0': 'a-GY', 'N0N0': 'a-YY', 'N0G0': 'a-YY', 'N0Y0': 'a-YY', 'N0S0': 'a-YY', 'G0N0': 'a-YY', 'G0G0': 'a-YY', 'G0Y0': 'a-YY', 'G0S0': 'a-YY', 'Y0N0': 'a-YY', 'Y0G0': 'a-YY', 'Y0Y0': 'a-YY', 'Y0S0': 'a-YY', 'S0N0': 'a-YY', 'S0G0': 'a-YY', 'S0Y0': 'a-YY', 'S0S0': 'a-YY', '00NN': 'a-N0', '00NG': 'a-G0', '00NY': 'a-G0', '00NS': 'a-G0', '00GN': 'a-G0', '00GG': 'a-G0', '00GY': 'a-G0', '00GS': 'a-G0', '00YN': 'a-G0', '00YG': 'a-G0', '00YY': 'a-G0', '00YS': 'a-G0', '00SN': 'a-G0', '00SG': 'a-G0', '00SY': 'a-G0', '00SS': 'a-G0', 'NN00': 'a-0N', 'NG00': 'a-0G', 'NY00': 'a-0G', 'NS00': 'a-0G', 'GN00': 'a-0G', 'GG00': 'a-0G', 'GY00': 'a-0G', 'GS00': 'a-0G', 'YN00': 'a-0G', 'YG00': 'a-0G', 'YY00': 'a-0G', 'YS00': 'a-0G', 'SN00': 'a-0G', 'SG00': 'a-0G', 'SY00': 'a-0G', 'SS00': 'a-0G', 'NNN0': 'a-0N', 'NNG0': 'a-0N', 'NNY0': 'a-0N', 'NNS0': 'a-0N', 'NGN0': 'a-0Y', 'NGG0':

In [115]:
dict_policyState["NYG0"]

'a-0Y'

In [116]:
print(policy_actualName)

['a-GG', 'a-YG', 'a-YG', 'a-YG', 'a-YG', 'a-GY', 'a-GY', 'a-GY', 'a-GY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-YY', 'a-N0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-G0', 'a-0N', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0G', 'a-0N', 'a-0N', 'a-0N', 'a-0N', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0N', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0N', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0N', 'a-0N', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0N', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-0Y', 'a-N0', 'a-Y0', 'a-Y0', 'a-Y0',

In [117]:
print(policy)

(5, 9, 9, 9, 9, 6, 6, 6, 6, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 22, 22, 22, 20, 22, 22, 22, 20, 22, 22, 22, 22, 22, 22, 20, 20, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 20, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 16, 18, 18, 18, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 16, 16, 16, 18, 16, 18, 18, 18, 16, 18, 18, 18, 18, 18, 18, 18, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 

In [118]:
dictionary1

{('0000', 'a-NN', 'N0N0'): {'probability': 'P1-0000-N0N0'},
 ('0000', 'a-NG', 'N0G0'): {'probability': 'P1-0000-N0G0'},
 ('0000', 'a-NY', 'N0Y0'): {'probability': 'P1-0000-N0Y0'},
 ('0000', 'a-NS', 'N0S0'): {'probability': 'P1-0000-N0S0'},
 ('0000', 'a-GN', 'G0N0'): {'probability': 'P1-0000-G0N0'},
 ('0000', 'a-GG', 'G0G0'): {'probability': 'P1-0000-G0G0'},
 ('0000', 'a-GY', 'G0Y0'): {'probability': 'P1-0000-G0Y0'},
 ('0000', 'a-GS', 'G0S0'): {'probability': 'P1-0000-G0S0'},
 ('0000', 'a-YN', 'Y0N0'): {'probability': 'P1-0000-Y0N0'},
 ('0000', 'a-YG', 'Y0G0'): {'probability': 'P1-0000-Y0G0'},
 ('0000', 'a-YY', 'Y0Y0'): {'probability': 'P1-0000-Y0Y0'},
 ('0000', 'a-YS', 'Y0S0'): {'probability': 'P1-0000-Y0S0'},
 ('0000', 'a-SN', 'S0N0'): {'probability': 'P1-0000-S0N0'},
 ('0000', 'a-SG', 'S0G0'): {'probability': 'P1-0000-S0G0'},
 ('0000', 'a-SY', 'S0Y0'): {'probability': 'P1-0000-S0Y0'},
 ('0000', 'a-SS', 'S0S0'): {'probability': 'P1-0000-S0S0'},
 ('0000', 'a-NN', '00N0'): {'probability

In [119]:
dictionary2

{'0000': 'a-NG',
 'N000': 'a-YG',
 'G000': 'a-YG',
 'Y000': 'a-YG',
 'S000': 'a-YG',
 '00N0': 'a-NY',
 '00G0': 'a-NY',
 '00Y0': 'a-NY',
 '00S0': 'a-NY',
 'N0N0': 'a-YY',
 'N0G0': 'a-YY',
 'N0Y0': 'a-YY',
 'N0S0': 'a-YY',
 'G0N0': 'a-YY',
 'G0G0': 'a-YY',
 'G0Y0': 'a-YY',
 'G0S0': 'a-YY',
 'Y0N0': 'a-YY',
 'Y0G0': 'a-YY',
 'Y0Y0': 'a-YY',
 'Y0S0': 'a-YY',
 'S0N0': 'a-YY',
 'S0G0': 'a-YY',
 'S0Y0': 'a-YY',
 'S0S0': 'a-YY',
 '00NN': 'a-N0',
 '00NG': 'a-N0',
 '00NY': 'a-N0',
 '00NS': 'a-N0',
 '00GN': 'a-N0',
 '00GG': 'a-N0',
 '00GY': 'a-N0',
 '00GS': 'a-N0',
 '00YN': 'a-N0',
 '00YG': 'a-N0',
 '00YY': 'a-N0',
 '00YS': 'a-N0',
 '00SN': 'a-N0',
 '00SG': 'a-N0',
 '00SY': 'a-N0',
 '00SS': 'a-N0',
 'NN00': 'a-0N',
 'NG00': 'a-0G',
 'NY00': 'a-0G',
 'NS00': 'a-0G',
 'GN00': 'a-0N',
 'GG00': 'a-0G',
 'GY00': 'a-0G',
 'GS00': 'a-0G',
 'YN00': 'a-0N',
 'YG00': 'a-0G',
 'YY00': 'a-0G',
 'YS00': 'a-0G',
 'SN00': 'a-0N',
 'SG00': 'a-0G',
 'SY00': 'a-0G',
 'SS00': 'a-0G',
 'NNN0': 'a-0N',
 'NNG0': 'a-0N

# Optimal Strategy

In [120]:
#Optimal Path
initial_state='0000'
current_state = initial_state
dictionary1=transitions #just for make the life easier
dictionary2=dict_policyState #just for make the life easier


initial_action=dict_policyState['0000']
    
# Keep track of the path
path = []


next_state=None
    
# Follow the path until the final state is reached
while next_state != 'Final':
    # Extract the next action based on the current state
    current_action = dictionary2[current_state]
        
    # Find the next state based on the current state and action
    #next_state = None
    for key,value in dictionary1.items():
        #print(key)
        #print(value)
        if key[0] == current_state and key[1] == current_action:
            next_state = key[2]
            #print(next_state)
            
            break
        
    # If next state is not found, exit the loop
    #if next_state is None:
    #    break
        
    # Add the next state and action to the path
    path.append((current_state, current_action))
    #print(path)
        
    # Update current state
    current_state = next_state
    #print(current_state)
path.append(('Final','a-F'))   #the last state

In [121]:
#path
path

[('0000', 'a-GG'), ('G0G0', 'a-YY'), ('GYGY', 'a-M'), ('Final', 'a-F')]

In [122]:
# optimal strategy
optimal_strategy=path[-2][0]
optimal_strategy

'GYGY'

# Just for Presentation

In [102]:
# illustration
insp_types= {'N': 'No', 'G': 'Gel' , 'Y': 'Bio', 'S': 'Seq'}

list_optimal_strategy = []
for char in optimal_strategy:
    if char in insp_types:
        list_optimal_strategy.append(insp_types[char])

opt={'Inspection A1':list_optimal_strategy[0],'Inspection A2':list_optimal_strategy[1], 'Inspection B1':list_optimal_strategy[2], 'Inspection B2':list_optimal_strategy[3]}

In [798]:
opt

{'Inspection A1': 'No',
 'Inspection A2': 'Bio',
 'Inspection B1': 'Gel',
 'Inspection B2': 'Gel'}

In [799]:

print("Cost:   {}".format(rounded_total_cost))

length = 70
print("-" * length)
print("-" * length)
print("-" * length)



print("{:<12}{:<22}{}\n\n \n{:<12}{:<22}{}\n\n{:>62}\n\n{:<12}{:<22}{}".format("","Non-conf Insp1", "non conf-Insp2"
                                                        ,"Line A: ",z_A1, z_A2
                                                      ,z3,
                                                      "Line B: ",  z_B1, z_B2))


length = 70
print("-" * length)

print("\n{:<12}{:<22}{}\n\n \n{:<12}{:<22}{}\n\n{:>62}\n\n{:<12}{:<22}{}".format("","Inspection1", "Inspection2"
                                                        ,"Line A: ",opt['Inspection A1'], opt['Inspection A2']
                                                      ,"Final Seq",
                                                      "Line B: ", opt['Inspection B1'], opt['Inspection B2']))

Cost:   -7810.27
----------------------------------------------------------------------
----------------------------------------------------------------------
----------------------------------------------------------------------
            Non-conf Insp1        non conf-Insp2

 
Line A:     0.1                   0.9

                                                           0.5

Line B:     0.8                   0.2
----------------------------------------------------------------------

            Inspection1           Inspection2

 
Line A:     No                    Bio

                                                     Final Seq

Line B:     Gel                   Gel


# Compare Different scenarion (Zs)

# Extra

In [ ]:
# def calculate_formula(string):
    
#     """
#     In tis part we get the probability of a transition like string='P1-0000-00G0' as an input.
#     Others are the default inputs
#     the output is the real number (real probability) for the transition.
    
#     """
#     alpha_A1, beta_A1, alpha_A2, beta_A2, alpha_B1, beta_B1, alpha_B2, beta_B2 , alpha_M, beta_M= get_alpha_beta(string) 
    
    
#     #initial values
#     value=0
    
    
#     #conditions
#     if string[:2] == "P1":
#         #P1
#         P1= (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         P_IA1p=P1
#         value = P1
        
#     elif string[:2] == "P2":
#         #P2
#         P2= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
#         P_IB1p=P2
#         value = P2
        
#     elif string[:2] == "P3":   
#         #P1
#         P1= (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         P_IA1p=P1
    
#         #P2
#         P2= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
#         P_IB1p=P2
        
#         #P3
#         P3=P_IA1p*P_IB1p
#         value = P3
        
        
#     elif string[:2] == "P4":
        
#         ##P_IA2p_given_IA1p
#         # Calculate the numerator and denominator separately
#         numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
#         denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         # Compute the fraction
#         fraction = numerator / denominator
#         # Add the last term
#         P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
#         #P4
#         P4= P_IA2p_given_IA1p
#         value = P4
        
#     elif string[:2] == "P5":
        
#         ##P_IB2p_given_IB1p
#         numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
#         denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1

#         # Compute the fraction
#         fraction2 = numerator2 / denominator2

#         # Add the last term
#         P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
#         #P5
#         P5=P_IB2p_given_IB1p
#         value = P5
        
        
        
#     elif string[:2] == "P6":
        
#         #P1
#         P1= (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         P_IA1p=P1
        
#         ##P_IB2p_given_IB1p
#         numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
#         denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1

#         # Compute the fraction
#         fraction2 = numerator2 / denominator2

#         # Add the last term
#         P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        
#         #P6
#         P6=P_IB2p_given_IB1p*P_IA1p
#         value = P6
        
        
        
#     elif string[:2] == "P7":
        
#         #P2
#         P2= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
#         P_IB1p=P2
        
#         ##P_IA2p_given_IA1p
#         # Calculate the numerator and denominator separately
#         numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
#         denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         # Compute the fraction
#         fraction = numerator / denominator
#         # Add the last term
#         P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
#         #P7
#         P7=P_IA2p_given_IA1p*P_IB1p
#         value = P7    
        
    
#     elif string[:2] == "P8":
        
#         ##P_IA2p_given_IA1p
#         # Calculate the numerator and denominator separately
#         numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
#         denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         # Compute the fraction
#         fraction = numerator / denominator
#         # Add the last term
#         P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
#         ##P_IB2p_given_IB1p
#         numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
#         denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1

#         # Compute the fraction
#         fraction2 = numerator2 / denominator2

#         # Add the last term
#         P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        
#         #P8
#         P8= P_IA2p_given_IA1p*P_IB2p_given_IB1p
#         value = P8
        
#     elif string[:2] == "P9":
        
#         #P_IB1p
#         P2= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
#         P_IB1p=P2
        
#         #P_IB2p_given_IB1p
#         numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
#         denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
#         # Compute the fraction
#         fraction2 = numerator2 / denominator2
#         # Add the last term
#         P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        
        
#         ##P_IB2p
#         P_IB2p=P_IB1p*P_IB2p_given_IB1p
        
        
        
        
#         #P_IA1p
#         P1= (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         P_IA1p=P1
        
        
#         #P_IA2p_given_IA1p
#         # Calculate the numerator and denominator separately
#         numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
#         denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#         # Compute the fraction
#         fraction = numerator / denominator
#         # Add the last term
#         P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
        
#         ##P_IA2p
#         P_IA2p=P_IA1p*P_IA2p_given_IA1p
        
        
        
#         #P9
#         term1_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 *z_A1* z_B1 * z_B2
#         term1_denominator = P_IA2p * P_IB2p
#         term1 = term1_numerator / term1_denominator

#         term2_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 * z_A1 * (z_B1 * (1 - z_B2) + (1 - z_B1) * z_B2)
#         term2_denominator = P_IA2p * P_IB2p
#         term2 = term2_numerator / term2_denominator

#         term3_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 * (1 - z_A1) * z_A2 * (z_B1 + (1 - z_B1) * z_B2)
#         term3_denominator = P_IA2p * P_IB2p
#         term3 = term3_numerator / term3_denominator

#         term4_numerator = beta_M * beta_A1 * beta_A2 * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (z_A1 + (1 - z_A1) * z_A2)
#         term4_denominator = P_IA2p * P_IB2p
#         term4 = term4_numerator / term4_denominator

#         term5_numerator = beta_M * beta_B1 * beta_B2 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) *(z_B1 +  (1 - z_B1) * z_B2)
#         term5_denominator = P_IA2p * P_IB2p
#         term5 = term5_numerator / term5_denominator

#         #10
#         term6_numerator = beta_M * z3 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2)
#         term6_denominator = P_IA2p * P_IB2p
#         term6 = term6_numerator / term6_denominator

        
#         #11
#         term7_numerator = (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (1 - alpha_M) * (1 - z3)
#         term7_denominator = P_IA2p * P_IB2p
#         term7 = term7_numerator / term7_denominator

#         # Calculate the total expression
#         P9 = term1 + term2 + term3 + term4 + term5 + term6 + term7
#         value = P9
            
#     elif string[:2] == "Pf":
        
#         ###P10=Pf
#         Pf=1
#         value = Pf
        
        
        
#     return value

In [ ]:
#old version
def calculate_formula(string):
    
    """
    In tis part we get the probability of a transition like string='P1-0000-00G0' as an input.
    Others are the default inputs
    the output is the real number (real probability) for the transition.
    
    """
    alpha_A1, beta_A1, alpha_A2, beta_A2, alpha_B1, beta_B1, alpha_B2, beta_B2 , alpha_M, beta_M= get_alpha_beta(string) 
    
    
    #initial values
    value=0
    
    #P(IA1p)
    P_IA1p=(1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
    #---------------------------------------------------
    #P_IA1f
    P_IA1f=1-P_IA1p
    #---------------------------------------------------
    #P_IB1p
    P_IB1p= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
    #---------------------------------------------------
    #P_IB1f
    P_IB1f=1-P_IB1p
    #---------------------------------------------------
    ##P_IA2p_given_IA1p
    numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
    denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
    # Compute the fraction
    fraction = numerator / denominator
    # Add the last term
    P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
    #---------------------------------------------------
    ## P_IA2f_given_IA1p
    P_IA2f_given_IA1p=1-P_IA2p_given_IA1p
    #---------------------------------------------------
    ##P_IB2p_given_IB1p
    numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
    denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
    # Compute the fraction
    fraction2 = numerator2 / denominator2
    # Add the last term
    P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
    #---------------------------------------------------
    #P_IB2f_given_IB1p
    P_IB2f_given_IB1p=1-P_IB2p_given_IB1p
    
    #conditions
    if string[:2] == "P1":
        #P1
        P1=P_IA1f*P_IB1p
        value = P1
        
    elif string[:2] == "P2":
        #P2
        P2=P_IA1p*P_IB1p
        value = P2
        
    elif string[:2] == "P3":   
        #P3
        P3=P_IA1p*P_IB1f
        value = P3
        
        
    elif string[:2] == "P4":
        
        ##P_IA2p_given_IA1p
        # Calculate the numerator and denominator separately
        numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
        denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        # Compute the fraction
        fraction = numerator / denominator
        # Add the last term
        P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
        #P4
        P4= P_IA2p_given_IA1p
        value = P4
        
    elif string[:2] == "P5":
        
        ##P_IB2p_given_IB1p
        numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
        denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1

        # Compute the fraction
        fraction2 = numerator2 / denominator2

        # Add the last term
        P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        #P5
        P5=P_IB2p_given_IB1p
        value = P5
        
        
        
    elif string[:2] == "P6":
        
        #P1
        P1= (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        P_IA1p=P1
        
        ##P_IB2p_given_IB1p
        numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
        denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1

        # Compute the fraction
        fraction2 = numerator2 / denominator2

        # Add the last term
        P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        
        #P6
        P6=P_IB2p_given_IB1p*P_IA1p
        value = P6
        
        
        
    elif string[:2] == "P7":
        
        #P2
        P2= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
        P_IB1p=P2
        
        ##P_IA2p_given_IA1p
        # Calculate the numerator and denominator separately
        numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
        denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        # Compute the fraction
        fraction = numerator / denominator
        # Add the last term
        P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
        #P7
        P7=P_IA2p_given_IA1p*P_IB1p
        value = P7    
        
    
    elif string[:2] == "P8":
        
        ##P_IA2p_given_IA1p
        # Calculate the numerator and denominator separately
        numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
        denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        # Compute the fraction
        fraction = numerator / denominator
        # Add the last term
        P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
        ##P_IB2p_given_IB1p
        numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
        denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1

        # Compute the fraction
        fraction2 = numerator2 / denominator2

        # Add the last term
        P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        
        #P8
        P8= P_IA2p_given_IA1p*P_IB2p_given_IB1p
        value = P8
        
    elif string[:2] == "P9":
        
        #P_IB1p
        P2= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
        P_IB1p=P2
        
        #P_IB2p_given_IB1p
        numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
        denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
        # Compute the fraction
        fraction2 = numerator2 / denominator2
        # Add the last term
        P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        
        
        ##P_IB2p
        P_IB2p=P_IB1p*P_IB2p_given_IB1p
        
        
        
        
        #P_IA1p
        P1= (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        P_IA1p=P1
        
        
        #P_IA2p_given_IA1p
        # Calculate the numerator and denominator separately
        numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
        denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        # Compute the fraction
        fraction = numerator / denominator
        # Add the last term
        P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        
        
        ##P_IA2p
        P_IA2p=P_IA1p*P_IA2p_given_IA1p
        
        
        
        #P9
        
        #6
        term1_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 *z_A1* z_B1 * z_B2
        term1_denominator = P_IA2p * P_IB2p
        term1 = term1_numerator / term1_denominator
        
        
        #3+4
        term2_numerator = beta_M * beta_A1 * beta_A2  * beta_B2 * z_A1 * ( beta_B1*z_B1 * (1 - z_B2) + (1 - alpha_B1)*(1 - z_B1) * z_B2)
        term2_denominator = P_IA2p * P_IB2p
        term2 = term2_numerator / term2_denominator

        #5+8
        term3_numerator = beta_M * (1 - alpha_A1)*  beta_A2 * beta_B1 * beta_B2 * (1 - z_A1) * z_A2 * (beta_B1*z_B1 + (1 - alpha_B1)*(1 - z_B1) * z_B2)
        term3_denominator = P_IA2p * P_IB2p
        term3 = term3_numerator / term3_denominator

        #1+9
        term4_numerator = beta_M *  beta_A2 * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (beta_A1 *z_A1 + (1 - alpha_A1)*(1 - z_A1) * z_A2)
        term4_denominator = P_IA2p * P_IB2p
        term4 = term4_numerator / term4_denominator

        #2+7
        term5_numerator = beta_M *  beta_B2 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) *(beta_B1 *z_B1 +  (1 - alpha_B1)*(1 - z_B1) * z_B2)
        term5_denominator = P_IA2p * P_IB2p
        term5 = term5_numerator / term5_denominator

        #11
        term6_numerator = beta_M * z3 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2)
        term6_denominator = P_IA2p * P_IB2p
        term6 = term6_numerator / term6_denominator

        
        #10
        term7_numerator =  (1 - alpha_M) * (1 - z3)*(1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) 
        term7_denominator = P_IA2p * P_IB2p
        term7 = term7_numerator / term7_denominator

        # Calculate the total expression
        P9 = term1 + term2 + term3 + term4 + term5 + term6 + term7
        value = P9
            
    elif string[:2] == "Pf":
        
        ###P10=Pf
        Pf=1
        value = Pf
        
        
        
    return value

# Pre-Probabilities

In [16]:
#Pre Probabilities
#P(IA1p)
P_IA1p=(1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
#---------------------------------------------------
#P_IA1f
P_IA1f=1-P_IA1p
#---------------------------------------------------
#P_IB1p
P_IB1p= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
#---------------------------------------------------
#P_IB1f
P_IB1f=1-P_IB1p
#---------------------------------------------------
##P_IA2p_given_IA1p
numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
# Compute the fraction
fraction = numerator / denominator
# Add the last term
P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
#---------------------------------------------------
## P_IA2f_given_IA1p
P_IA2f_given_IA1p=1-P_IA2p_given_IA1p
#---------------------------------------------------
##P_IB2p_given_IB1p
numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
# Compute the fraction
fraction2 = numerator2 / denominator2
# Add the last term
P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
#---------------------------------------------------
#P_IB2f_given_IB1p
P_IB2f_given_IB1p=1-P_IB2p_given_IB1p

In [ ]:
#Pre-Probabilities 2
###########  1-From State = G0000   taking  action=a-AB  ###########
#P1=P(IA1f)*P(IB1p)
P1=P_IA1f*P_IB1p
#---------------------------------------------------
#P2= P(IA1p)*P(IB1p)
P2=P_IA1p*P_IB1p
#---------------------------------------------------
#P3=P(IA1p)*P(IB1f)
P3=P_IA1p*P_IB1f
#---------------------------------------------------
#P4=P(IA1f)*P(IB1f)
P4=P_IA1f*P_IB1f
#---------------------------------------------------


###########  2-From State    G0010     taking    action=a-AB  ###########
#P5= P(IA1p)*P(IB2p|IB1p)
P5=P_IA1p*P_IB2p_given_IB1p
#---------------------------------------------------
#P6= P(IA1f)*P(IB2p|IB1p)
P6=P_IA1f*P_IB2p_given_IB1p
#---------------------------------------------------
#P7= P(IA1p)*P(IB2f|IB1p)
P7=P_IA1p*P_IB2f_given_IB1p
#---------------------------------------------------
#P8= P(IA1f)*P(IB2f|IB1p)
P8=P_IA1f*P_IB2f_given_IB1p
#---------------------------------------------------


###########  3-From state   G1010   taking action=a-AB  ###########
#P9= P(IA2f|IA1p)*P(IB2p|IB1p)
P9=P_IA2f_given_IA1p*P_IB2p_given_IB1p
#---------------------------------------------------
#P10= P(IA2p|IA1p)*P(IB2p|IB1p)
P10=P_IA2p_given_IA1p*P_IB2p_given_IB1p
#---------------------------------------------------
#P11= P(IA2p|IA1p)*P(IB2f|IB1p)
P11=P_IA2p_given_IA1p*P_IB2f_given_IB1p
#---------------------------------------------------
#P12= P(IA2f|IA1p)*P(IB2f|IB1p)
P12=P_IA2f_given_IA1p*P_IB2f_given_IB1p
#---------------------------------------------------


###########  4-From state   G1000   taking action=a-AB  ###########
#P13=P(IA2p|IA1p)*P(IB1p)
P13=P_IA2p_given_IA1p*P_IB1p
#---------------------------------------------------
#P14=P(IA2p|IA1p)*P(IB1f)
P14=P_IA2p_given_IA1p*P_IB1f
#---------------------------------------------------
#P15=P(IA2f|IA1p)*P(IB1p)
P15=P_IA2f_given_IA1p*P_IB1p
#---------------------------------------------------
#P16=P(IA2f|IA1p)*P(IB1f)
P16=P_IA2f_given_IA1p*P_IB1f
#---------------------------------------------------


###########  5-From state   G0011   taking action=a-A0  ###########
#P17=P(IA1p)
P17=P_IA1p
#---------------------------------------------------
#P18=P(IA1f)
P18=P_IA1f
#---------------------------------------------------


###########  6-From state G1100 taking action a-0B  ###########
#P19=P(IB1p)
P19=P_IB1p
#---------------------------------------------------
#P20=P(IB1f)
P20=P_IB1f
#---------------------------------------------------


###########  7-From state   G1011   taking action=a-A0  ###########
#P21=P(IA2p|IA1p)
P21=P_IA2p_given_IA1p
#---------------------------------------------------
#P22= P(IA2f|IA1p)
P22=P_IA2f_given_IA1p
#---------------------------------------------------


###########  8-From state G1110 taking action a-0B  ###########
#P23=P(IB2p|IB1p)
P23=P_IB2p_given_IB1p
#P24=P(IB2f|IB1p)
P24=P_IB2f_given_IB1p


###########  9-From State G1111 taking action a-M  ###########
#P25
#6
term1_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 *z_A1* z_B1 * z_B2
term1_denominator = P_IA2p * P_IB2p
term1 = term1_numerator / term1_denominator
        
        
#3+4
term2_numerator = beta_M * beta_A1 * beta_A2  * beta_B2 * z_A1 * ( beta_B1*z_B1 * (1 - z_B2) + (1 - alpha_B1)*(1 - z_B1) * z_B2)
term2_denominator = P_IA2p * P_IB2p
term2 = term2_numerator / term2_denominator

#5+8
term3_numerator = beta_M * (1 - alpha_A1)*  beta_A2 * beta_B1 * beta_B2 * (1 - z_A1) * z_A2 * (beta_B1*z_B1 + (1 - alpha_B1)*(1 - z_B1) * z_B2)
term3_denominator = P_IA2p * P_IB2p
term3 = term3_numerator / term3_denominator

#1+9
term4_numerator = beta_M *  beta_A2 * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (beta_A1 *z_A1 + (1 - alpha_A1)*(1 - z_A1) * z_A2)
term4_denominator = P_IA2p * P_IB2p
term4 = term4_numerator / term4_denominator

#2+7
term5_numerator = beta_M *  beta_B2 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) *(beta_B1 *z_B1 +  (1 - alpha_B1)*(1 - z_B1) * z_B2)
term5_denominator = P_IA2p * P_IB2p
term5 = term5_numerator / term5_denominator

#11
term6_numerator = beta_M * z3 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2)
term6_denominator = P_IA2p * P_IB2p
term6 = term6_numerator / term6_denominator

        
#10
term7_numerator =  (1 - alpha_M) * (1 - z3)*(1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) 
term7_denominator = P_IA2p * P_IB2p
term7 = term7_numerator / term7_denominator

# Calculate the total expression
P25 = term1 + term2 + term3 + term4 + term5 + term6 + term7
#---------------------------------------------------
#P26=1-P25
P26=1-P25



###########  10-From state GF taking action a-F  ###########
#P27=1
P27=1

In [331]:
#test

# single_trans={('Final', 'a-F', 'Final'): {'probability': 'P13-G000-GYG0'}}
# inspections_of_transition = {}
# insp_A1='0'
# insp_A2='0'
# insp_B1='0'
# insp_B2='0'
# insp_M='0'
# on_line_A=True
# on_line_B=True
# on_line_M=True

# for key, value in single_trans.items():
#     from_state, action, to_state = key
#     #print(from_state)
#     #print(action)
#     line_A_inspection_1 = from_state[0]
#     line_A_inspection_2 = from_state[1]
#     line_B_inspection_1 = from_state[2]
#     line_B_inspection_2 = from_state[3]
#     line_M_inspection=insp_M
    
#     if action!='a-M':
#         while on_line_A==True:
#             if line_A_inspection_1 == '0':
#                  # If no inspection indicated for line A
#                 insp_A1 = action.split('-')[1][0]
#                 insp_A2='0'
#                 on_line_A=False

#             elif line_A_inspection_1 != '0': #and action!='a-M':

#                 insp_A1=line_A_inspection_1
#                 insp_A2=action.split('-')[1][0]
#                 on_line_A=False




#         while on_line_B==True:
#             if line_B_inspection_1 == '0':
#                  # If no inspection indicated for line A
#                 insp_B1 = action.split('-')[1][1]
#                 on_line_B=False

#             elif line_A_inspection_1 != '0': #and action!='a-M':

#                 insp_B1=line_B_inspection_1
#                 insp_B2=action.split('-')[1][1]
#                 on_line_B=False
#     else:   #if action =='a-M':
#         insp_A1=line_A_inspection_1
#         insp_A2=line_A_inspection_2
#         insp_B1=line_B_inspection_1
#         insp_B2=line_B_inspection_2
#         insp_M='S'
        
# #     while on_line_M==True:
# #     #print(insp_A2)
# #         if action =='a-M':
# #             line_M_inspection='S'
# #             on_line_M=False
# #         else:
# #             on_line_M=False


# inspections_of_transition = {
#                 'line_A_inspection_1': insp_A1,
#                 'line_A_inspection_2': insp_A2,

#                 'line_B_inspection_1': insp_B1,
#                 'line_B_inspection_2': insp_B2,
    
#                 'line_M_inspection':   insp_M
#         }
# dict_of_alphas_betas={}
# alpha_beta_mapping = {'N': (alpha_N, beta_N), 'G': (alpha_G, beta_G), 'Y': (alpha_Y, beta_Y), 'S': (alpha_S, beta_S)}
# for key, value in inspections_of_transition.items():
    
#     alpha_value, beta_value = alpha_beta_mapping.get(value, (None, None))
#     dict_of_alphas_betas[key] = (alpha_value, beta_value)

# dict_of_alphas_betas

In [ ]:
#old version
def calculate_formula(key,string):
    
    """
    It is used in create_P() function
    it gets key and value from transitions , which include all transitions.
    Thsi function uses dict_of_alphas_betas inside itself.
    It gives us the value (float number) for the probability of a transition.
    """
    
    
    #initial values
    value=0
    
    #get all alphas and betas
    dict_of_alphas_betas=get_alpha_beta(key)
    
    alpha_A1=dict_of_alphas_betas['line_A_inspection_1'][0]
    beta_A1=dict_of_alphas_betas['line_A_inspection_1'][1]


    alpha_A2=dict_of_alphas_betas['line_A_inspection_2'][0]
    beta_A2=dict_of_alphas_betas['line_A_inspection_2'][1]


    alpha_B1=dict_of_alphas_betas['line_B_inspection_1'][0]
    beta_B1=dict_of_alphas_betas['line_B_inspection_1'][1]


    alpha_B2=dict_of_alphas_betas['line_B_inspection_2'][0]
    beta_B2=dict_of_alphas_betas['line_B_inspection_2'][1]


    alpha_M=dict_of_alphas_betas['line_M_inspection'][0]
    beta_M=dict_of_alphas_betas['line_M_inspection'][1]
    
    
    #the transition which has happend on all states which are before the final state.
    if beta_M!='unknown':
    
    
        
        
        #the below proability are the main formula for the probabilities.
        #P(IA1p)
        P_IA1p=(1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        #---------------------------------------------------
        #P_IA1f
        P_IA1f=1-P_IA1p
        #---------------------------------------------------
        #P_IB1p
        P_IB1p= (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
        #---------------------------------------------------
        #P_IB1f
        P_IB1f=1-P_IB1p
        #---------------------------------------------------
        ##P_IA2p_given_IA1p
        numerator = (1 - alpha_A1) * (1 - z_A1) * (1 - alpha_A2) * (1 - z_A2) + beta_A1 * z_A1 * beta_A2 * (1-z_A2)
        denominator = (1 - alpha_A1) * (1 - z_A1) + beta_A1 * z_A1
        # Compute the fraction
        fraction = numerator / denominator
        # Add the last term
        P_IA2p_given_IA1p = fraction + beta_A2 * z_A2
        #---------------------------------------------------
        ## P_IA2f_given_IA1p
        P_IA2f_given_IA1p=1-P_IA2p_given_IA1p
        #---------------------------------------------------
        ##P_IB2p_given_IB1p
        numerator2 = (1 - alpha_B1) * (1 - z_B1) * (1 - alpha_B2) * (1 - z_B2) + beta_B1 * z_B1 * beta_B2 * (1-z_B2)
        denominator2 = (1 - alpha_B1) * (1 - z_B1) + beta_B1 * z_B1
        # Compute the fraction
        fraction2 = numerator2 / denominator2
        # Add the last term
        P_IB2p_given_IB1p = fraction2 + beta_B2 * z_B2
        #---------------------------------------------------
        #P_IB2f_given_IB1p
        P_IB2f_given_IB1p=1-P_IB2p_given_IB1p
        #---------------------------------------------------
        #P_IA2p
        P_IA2p=P_IA1p*P_IA2p_given_IA1p
        #---------------------------------------------------
        #P_IB2p
        P_IB2p=P_IB1p*P_IB2p_given_IB1p

        #conditions, the proablity type of a transition., we use this
        #to indicate that for the current transition which above formula we need to use.
        if string[:3] == "P1-":
            #P1
            P1=P_IA1p*P_IB1p
            value = P1

        elif string[:3] == "P2-":
            #P2
            P2=P_IA1f*P_IB1p
            value = P2

        elif string[:3] == "P3-":   
            #P3
            P3=P_IA1p*P_IB1f
            value = P3


        elif string[:3] == "P4-":
            #P4
            P4=P_IA1f*P_IB1f
            value = P4

        elif string[:3] == "P5-":
            #P5
            P5=P_IA1p*P_IB2p_given_IB1p
            value = P5



        elif string[:3] == "P6-":
            #P6
            P6=P_IA1f*P_IB2p_given_IB1p
            value = P6



        elif string[:3] == "P7-":
            #P7
            P7=P_IA1p*P_IB2f_given_IB1p
            value = P7    


        elif string[:3] == "P8-":
            #P8
            P8=P_IA1f*P_IB2f_given_IB1p
            value = P8



        elif string[:3] == "P9-":  
            #P9
            P9=P_IA2p_given_IA1p*P_IB2p_given_IB1p
            value=P9



        elif string[:3] == "P10":
            #P10
            P10=P_IA2f_given_IA1p*P_IB2p_given_IB1p
            value=P10


        elif string[:3] == "P11":
            #P11
            P11=P_IA2p_given_IA1p*P_IB2f_given_IB1p
            value=P11    


        elif string[:3] == "P12":
            #P12
            P12=P_IA2f_given_IA1p*P_IB2f_given_IB1p
            value=P12    



        elif string[:3] == "P13":
            #P13
            P13=P_IA2p_given_IA1p*P_IB1p
            value=P13   


        elif string[:3] == "P14":
            #P14
            P14=P_IA2p_given_IA1p*P_IB1f
            value=P14



        elif string[:3] == "P15":
            #P15
            P15=P_IA2f_given_IA1p*P_IB1p
            value=P15   


        elif string[:3] == "P16":
            #P16
            P16=P_IA2f_given_IA1p*P_IB1f
            value=P16


        elif string[:3] == "P17":
            #P17
            P17=P_IA1p
            value=P17    



        elif string[:3] == "P18":
            #P18
            P18=P_IA1f
            value=P18

        elif string[:3] == "P19":
            #P19
            P19=P_IB1p
            value=P19


        elif string[:3] == "P20":
            #P20
            P20=P_IB1f
            value=P20

        elif string[:3] == "P21":
            #P21
            P21=P_IA2p_given_IA1p
            value=P21    


        elif string[:3] == "P22":
            #P22
            P22=P_IA2f_given_IA1p
            value=P22

        elif string[:3] == "P23":
            #P23
            P23=P_IB2p_given_IB1p
            value=P23

        elif string[:3] == "P24":
            #P24
            P24=P_IB2f_given_IB1p
            value=P24    

        #P(I3p|IAp, IBp)
        elif string[:3] == "P25":
            #P25
            #6
            term1_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 *z_A1* z_B1 * z_B2
            term1_denominator = P_IA2p * P_IB2p
            term1 = term1_numerator / term1_denominator


            #3+4
            term2_numerator = beta_M * beta_A1 * beta_A2  * beta_B2 * z_A1 * ( beta_B1*z_B1 * (1 - z_B2) + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term2_denominator = P_IA2p * P_IB2p
            term2 = term2_numerator / term2_denominator

            #5+8
            term3_numerator = beta_M * (1 - alpha_A1)*  beta_A2 * beta_B1 * beta_B2 * (1 - z_A1) * z_A2 * (beta_B1*z_B1 + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term3_denominator = P_IA2p * P_IB2p
            term3 = term3_numerator / term3_denominator

            #1+9
            term4_numerator = beta_M *  beta_A2 * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (beta_A1 *z_A1 + (1 - alpha_A1)*(1 - z_A1) * z_A2)
            term4_denominator = P_IA2p * P_IB2p
            term4 = term4_numerator / term4_denominator

            #2+7
            term5_numerator = beta_M *  beta_B2 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) *(beta_B1 *z_B1 +  (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term5_denominator = P_IA2p * P_IB2p
            term5 = term5_numerator / term5_denominator

            #11
            term6_numerator = beta_M * z3 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2)
            term6_denominator = P_IA2p * P_IB2p
            term6 = term6_numerator / term6_denominator


            #10
            term7_numerator =  (1 - alpha_M) * (1 - z3)*(1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) 
            term7_denominator = P_IA2p * P_IB2p
            term7 = term7_numerator / term7_denominator

            # Calculate the total expression
            P25 = term1 + term2 + term3 + term4 + term5 + term6 + term7
            value = P25

        elif string[:3] == "P26":
            #P26
            #6
            term1_numerator = beta_M * beta_A1 * beta_A2 * beta_B1 * beta_B2 *z_A1* z_B1 * z_B2
            term1_denominator = P_IA2p * P_IB2p
            term1 = term1_numerator / term1_denominator


            #3+4
            term2_numerator = beta_M * beta_A1 * beta_A2  * beta_B2 * z_A1 * ( beta_B1*z_B1 * (1 - z_B2) + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term2_denominator = P_IA2p * P_IB2p
            term2 = term2_numerator / term2_denominator

            #5+8
            term3_numerator = beta_M * (1 - alpha_A1)*  beta_A2 * beta_B1 * beta_B2 * (1 - z_A1) * z_A2 * (beta_B1*z_B1 + (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term3_denominator = P_IA2p * P_IB2p
            term3 = term3_numerator / term3_denominator

            #1+9
            term4_numerator = beta_M *  beta_A2 * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) * (beta_A1 *z_A1 + (1 - alpha_A1)*(1 - z_A1) * z_A2)
            term4_denominator = P_IA2p * P_IB2p
            term4 = term4_numerator / term4_denominator

            #2+7
            term5_numerator = beta_M *  beta_B2 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) *(beta_B1 *z_B1 +  (1 - alpha_B1)*(1 - z_B1) * z_B2)
            term5_denominator = P_IA2p * P_IB2p
            term5 = term5_numerator / term5_denominator

            #11
            term6_numerator = beta_M * z3 * (1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2)
            term6_denominator = P_IA2p * P_IB2p
            term6 = term6_numerator / term6_denominator


            #10
            term7_numerator =  (1 - alpha_M) * (1 - z3)*(1 - alpha_A1) * (1 - alpha_A2) * (1 - z_A1) * (1 - z_A2) * (1 - alpha_B1) * (1 - alpha_B2) * (1 - z_B1) * (1 - z_B2) 
            term7_denominator = P_IA2p * P_IB2p
            term7 = term7_numerator / term7_denominator

            # Calculate the total expression
            P25 = term1 + term2 + term3 + term4 + term5 + term6 + term7

            P26=1-P25
            value = P26
            
    #the transition which happens on the final state  
    elif beta_M!='unknown':
    
        #initial values
        value=0
        #string[:3] == "P27":
        #P27
        P27=1
        value=P27
        
    return round(value,3)

In [306]:
# import pandas as pd

# # Load the Excel file
# #file_path = 'path_to_your_file.xlsx'
# #df = pd.read_excel(file_path)
# #df[df.sum(axis=1) != 1]

# # Filter rows where the sum is not 1 and select rows and the sum column
# filtered_rows = P[P.sum(axis=1) != 1]

# Calculate the sum of each row
#filtered_rows['Row_Sum'] = filtered_rows.sum(axis=1)
#print(filtered_rows['Row_Sum'])
# filtered_rows.to_excel("filtered rows.xlsx")
# P.to_excel("trans prob30.xlsx")